In [1]:
import sys
sys.path.append("../")
import pandas as pd
import scraping.forexfactory_calendar as ff

import pandas as pd
from dateutil import parser
from sklearn.preprocessing import StandardScaler

import re
import numpy as np
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)
import re
import numpy as np
import pandas as pd

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder


In [2]:
forexfox = ff.get_monthly_data("jan.2025").copy()

forexfox_an = forexfox.copy()
print(forexfox_an.head()) 



Fetching data for: jan.2025
        Date     Time Currency                 Event Actual Forecast Previous
0  Wed Jan 1  All Day      CHF          Bank Holiday                         
1  Wed Jan 1  All Day      EUR   French Bank Holiday                         
2  Wed Jan 1  All Day      EUR   German Bank Holiday                         
3  Wed Jan 1  All Day      EUR  Italian Bank Holiday                         
4  Wed Jan 1   3:00am      GBP          Bank Holiday                         


In [3]:
forexfox_an.head(75)
forexfox_an.dtypes


Date        object
Time        object
Currency    object
Event       object
Actual      object
Forecast    object
Previous    object
dtype: object

In [4]:


# List of symbols to remove
SYMBOLS_TO_REMOVE = [r'%', r'M', r'¥', r'\$', r'€', r'C\$', r'K', r'B', r'A', r'\s', r',']
pattern = '|'.join(SYMBOLS_TO_REMOVE)

# Define the standalone function
def remove_symbols(column):
    """Remove specified symbols from a column, handle multi-values, and convert to float."""
    cleaned_column = []
    
    for value in column:
        # Convert the value to a string and remove symbols using regex
        cleaned_value = re.sub(pattern, '', str(value))
        
        # Handle cases with multiple values separated by '|'
        if '|' in cleaned_value:
            # Split by '|' and take the first part
            cleaned_value = cleaned_value.split('|')[0]
        
        # Replace empty strings with NaN
        if cleaned_value == '':
            cleaned_column.append(np.nan)
        else:
            try:
                cleaned_column.append(float(cleaned_value))
            except ValueError:
                # If conversion to float fails, set as NaN
                cleaned_column.append(np.nan)
    
    # Return the cleaned values as a Pandas Series
    return pd.Series(cleaned_column)



# Apply `remove_symbols` to the specified columns
forexfox_an['Actual'] = remove_symbols(forexfox_an['Actual'])
forexfox_an['Previous'] = remove_symbols(forexfox_an['Previous'])
forexfox_an['Forecast'] = remove_symbols(forexfox_an['Forecast'])
# Keep only rows where at least one of 'Actual', 'Forecast', or 'Previous' is not NaN
# forexfox_an.dropna(subset=['Actual', 'Forecast', 'Previous'], how='all', inplace=True)


# Display the cleaned DataFrame
print(forexfox_an.head(75))





         Date     Time Currency                              Event  Actual  \
0   Wed Jan 1  All Day      CHF                       Bank Holiday     NaN   
1   Wed Jan 1  All Day      EUR                French Bank Holiday     NaN   
2   Wed Jan 1  All Day      EUR                German Bank Holiday     NaN   
3   Wed Jan 1  All Day      EUR               Italian Bank Holiday     NaN   
4   Wed Jan 1   3:00am      GBP                       Bank Holiday     NaN   
5   Wed Jan 1  All Day      CAD                       Bank Holiday     NaN   
6   Wed Jan 1  All Day      USD                       Bank Holiday     NaN   
7   Wed Jan 1  All Day      NZD                       Bank Holiday     NaN   
8   Wed Jan 1  All Day      JPY                       Bank Holiday     NaN   
9   Wed Jan 1   8:45pm      CNY           Caixin Manufacturing PMI   50.50   
10  Thu Jan 2  12:30am      AUD               Commodity Prices y/y  -10.70   
11  Thu Jan 2  All Day      CHF                       Bank Holid

In [5]:

def calculate_rate_of_change(forexfox_an):
    """Calculate rate of change between actual, previous, and forecast values."""
    
    # Ensure the numeric conversion
    # forexfox_an['Actual'] = pd.to_numeric(forexfox_an['Actual'], errors='coerce')
    # forexfox_an['Previous'] = pd.to_numeric(forexfox_an['Previous'], errors='coerce')
    # forexfox_an['Forecast'] = pd.to_numeric(forexfox_an['Forecast'], errors='coerce')

    # # Perform calculations
    forexfox_an['rate_change_actual_previous'] = (
        (forexfox_an['Actual'] - forexfox_an['Previous']) / forexfox_an['Previous']
    )

    forexfox_an['rate_change_actual_forecast'] = (
        (forexfox_an['Actual'] - forexfox_an['Forecast']) / forexfox_an['Forecast']
    )

    return forexfox_an
# Run function
forexfox_an = calculate_rate_of_change(forexfox_an)


In [6]:
forexfox_an.head(75)


,Date,Time,Currency,Event,Actual,Forecast,Previous,rate_change_actual_previous,rate_change_actual_forecast
0,Wed Jan 1,All Day,CHF,Bank Holiday,NaN,NaN,NaN,NaN,NaN
1,Wed Jan 1,All Day,EUR,French Bank Holiday,NaN,NaN,NaN,NaN,NaN
2,Wed Jan 1,All Day,EUR,German Bank Holiday,NaN,NaN,NaN,NaN,NaN
3,Wed Jan 1,All Day,EUR,Italian Bank Holiday,NaN,NaN,NaN,NaN,NaN
4,Wed Jan 1,3:00am,GBP,Bank Holiday,NaN,NaN,NaN,NaN,NaN
5,Wed Jan 1,All Day,CAD,Bank Holiday,NaN,NaN,NaN,NaN,NaN
6,Wed Jan 1,All Day,USD,Bank Holiday,NaN,NaN,NaN,NaN,NaN
7,Wed Jan 1,All Day,NZD,Bank Holiday,NaN,NaN,NaN,NaN,NaN
8,Wed Jan 1,All Day,JPY,Bank Holiday,NaN,NaN,NaN,NaN,NaN
9,Wed Jan 1,8:45pm,CNY,Caixin Manufacturing PMI,50.50,51.6,51.50,-0.019417,-0.021318


In [7]:
date_time_list = []
for index, row in forexfox_an.iterrows():
    # Check if 'Time' is set to "All Day"
    time_value = row['Time'] if row['Time'] != 'All Day' else '12:00am'
    date_string = f"{row['Date']} {time_value}"
    
    try:
        parsed_date = parser.parse(date_string)
    except parser.ParserError:
        print(f"Skipping invalid date format: {date_string} {index}")
        parsed_date = None
    
    date_time_list.append(parsed_date)

# Assign the parsed date-time values to a new 'DateTime' column
forexfox_an['DateTime'] = date_time_list





Skipping invalid date format: Mon Jan 20 Day 1 237
Skipping invalid date format: Tue Jan 21 Day 2 246
Skipping invalid date format: Wed Jan 22 Day 3 259
Skipping invalid date format: Thu Jan 23 Day 4 267
Skipping invalid date format: Fri Jan 24 Day 5 292


In [8]:
forexfox_an.head()

,Date,Time,Currency,Event,Actual,Forecast,Previous,rate_change_actual_previous,rate_change_actual_forecast,DateTime
0,Wed Jan 1,All Day,CHF,Bank Holiday,NaN,NaN,NaN,NaN,NaN,2025-01-01 00:00:00
1,Wed Jan 1,All Day,EUR,French Bank Holiday,NaN,NaN,NaN,NaN,NaN,2025-01-01 00:00:00
2,Wed Jan 1,All Day,EUR,German Bank Holiday,NaN,NaN,NaN,NaN,NaN,2025-01-01 00:00:00
3,Wed Jan 1,All Day,EUR,Italian Bank Holiday,NaN,NaN,NaN,NaN,NaN,2025-01-01 00:00:00
4,Wed Jan 1,3:00am,GBP,Bank Holiday,NaN,NaN,NaN,NaN,NaN,2025-01-01 03:00:00


In [9]:
print(forexfox_an)


           Date     Time Currency                                 Event  \
0     Wed Jan 1  All Day      CHF                          Bank Holiday   
1     Wed Jan 1  All Day      EUR                   French Bank Holiday   
2     Wed Jan 1  All Day      EUR                   German Bank Holiday   
3     Wed Jan 1  All Day      EUR                  Italian Bank Holiday   
4     Wed Jan 1   3:00am      GBP                          Bank Holiday   
5     Wed Jan 1  All Day      CAD                          Bank Holiday   
6     Wed Jan 1  All Day      USD                          Bank Holiday   
7     Wed Jan 1  All Day      NZD                          Bank Holiday   
8     Wed Jan 1  All Day      JPY                          Bank Holiday   
9     Wed Jan 1   8:45pm      CNY              Caixin Manufacturing PMI   
10    Thu Jan 2  12:30am      AUD                  Commodity Prices y/y   
11    Thu Jan 2  All Day      CHF                          Bank Holiday   
12    Thu Jan 2   2:00am 

In [10]:
from pytz import timezone  

# def enhance_datetime_features(df):
#     """Create comprehensive time-based features"""
#     df['DateTime'] = pd.to_datetime(df['DateTime'], utc=True)
    
#     # Convert to trading session timezones
#     df['NY_Time'] = df['DateTime'].dt.tz_convert(timezone('America/New_York'))
#     df['London_Time'] = df['DateTime'].dt.tz_convert(timezone('Europe/London'))
#     df['Tokyo_Time'] = df['DateTime'].dt.tz_convert(timezone('Asia/Tokyo'))
    
#     # # Enhanced temporal features
#     # df['Trading_Session'] = df['London_Time'].apply(get_trading_session)
#     # df['Days_Until_Next_FOMC'] = df['DateTime'].apply(days_until_fomc)
#     # df['Month_End'] = df['DateTime'].apply(lambda x: x.day == calendar.monthrange(x.year, x.month)[1])
    
#     return df

# print(enhance_datetime_features(forexfox_an))


In [11]:
# Step 3: Feature Engineering - Extract useful information from 'DateTime'
forexfox_an['DayOfWeek'] = forexfox_an['DateTime'].dt.dayofweek
forexfox_an['Hour'] = forexfox_an['DateTime'].dt.hour

# Step 4: Drop original 'Date', 'Time', and 'DateTime' columns
forexfox_an = forexfox_an.drop(['DateTime'], axis=1)
forexfox_an = forexfox_an.drop(['Time'], axis=1)
forexfox_an = forexfox_an.drop(['Date'], axis=1)

# # Drop rows where 'DateTime' could not be parsed
# forexfox.dropna(subset=['DateTime'], inplace=True)


In [12]:
forexfox_an.replace([np.inf, -np.inf], np.nan, inplace=True)

# Keep only rows where at least one of 'Actual', 'Forecast', or 'Previous' is not NaN
forexfox_an.dropna(subset=['Actual', 'Forecast', 'Previous'], how='all', inplace=True)



# Display the cleaned DataFrame
print(forexfox_an.head(40))




   Currency                           Event  Actual  Forecast  Previous  \
9       CNY        Caixin Manufacturing PMI    50.5      51.6      51.5   
10      AUD            Commodity Prices y/y   -10.7       NaN     -10.8   
12      GBP              Nationwide HPI m/m     0.7       0.1       1.2   
13      EUR       Spanish Manufacturing PMI    53.3      53.6      53.1   
14      EUR       Italian Manufacturing PMI    46.2      44.9      44.5   
15      EUR  French Final Manufacturing PMI    41.9      41.9      41.9   
16      EUR  German Final Manufacturing PMI    42.5      42.5      42.5   
17      EUR         Final Manufacturing PMI    45.1      45.2      45.2   
18      EUR             M3 Money Supply y/y     3.8       3.5       3.4   
19      EUR               Private Loans y/y     0.9       0.9       0.8   
20      GBP         Final Manufacturing PMI    47.0      47.3      47.3   
21      USD             Unemployment Claims   211.0     222.0     220.0   
22      CAD              

In [13]:
le_currency = LabelEncoder()
le_event = LabelEncoder()
forexfox_an['Currency'] = le_currency.fit_transform(forexfox_an['Currency'])
forexfox_an['Event'] = le_event.fit_transform(forexfox_an['Event'])

In [14]:
imputer = KNNImputer(n_neighbors=3)
forexfox_an_imputed = pd.DataFrame(imputer.fit_transform(forexfox_an), columns=forexfox_an.columns)

print(forexfox_an_imputed)

     Currency  Event  Actual     Forecast  Previous  \
0         3.0   33.0   50.50    51.600000     51.50   
1         0.0   39.0  -10.70    -4.566667    -10.80   
2         5.0  165.0    0.70     0.100000      1.20   
3         4.0  209.0   53.30    53.600000     53.10   
4         4.0  133.0   46.20    44.900000     44.50   
5         4.0   81.0   41.90    41.900000     41.90   
6         4.0   97.0   42.50    42.500000     42.50   
7         4.0   69.0   45.10    45.200000     45.20   
8         4.0  146.0    3.80     3.500000      3.40   
9         4.0  189.0    0.90     0.900000      0.80   
10        5.0   69.0   47.00    47.300000     47.30   
11        8.0  221.0  211.00   222.000000    220.00   
12        1.0  152.0   52.20    51.900000     52.00   
13        8.0   69.0   49.40    48.300000     48.30   
14        8.0   43.0    0.00     0.300000      0.50   
15        8.0   55.0   -1.20    -2.400000     -4.20   
16        4.0  211.0  -25.30   -46.500000    -16.00   
17        

In [ ]:
# reference_date = pd.to_datetime("2024-11-01")

# def reconstruct_datetime(row):
#     # Calculate the actual date using reference_date and 'DayOfWeek'
#     adjusted_date = reference_date + pd.to_timedelta((row['DayOfWeek'] - reference_date.weekday()) % 7, unit='D')
#     return pd.Timestamp(adjusted_date.year, adjusted_date.month, adjusted_date.day, int(row['Hour']), 0)

# # Apply the function to reconstruct the DateTime column
# forexfox_an_imputed['DateTime'] = forexfox_an_imputed.apply(reconstruct_datetime, axis=1)

# # Step 9: Drop feature-engineered columns if they are not needed
# forexfox_an_imputed = forexfox_an_imputed.drop(['DayOfWeek', 'Hour'], axis=1)

# # Display the cleaned and imputed DataFrame
# print(forexfox_an_imputed.head(40))



NameError: name 'pd' is not defined